# 계층적 위험 등가(Hierarchical Risk Parity) 모형 투자자 성향별 포트폴리오 산출

In [34]:
import numpy as np
import pandas as pd
import cvxopt as opt
import datetime as dt
import scipy.cluster.hierarchy as sch
from cvxopt import blas, solvers
from datetime import timedelta
from pylab import *

# 예스트레이더 출력 결과물을 Pandas 형태로 포맷 변환

In [35]:
# 예스트레이더 종목검색 결과를 날짜 기준으로 포맷팅
def convert_yes_price_file(gubun):
    # 우리나라 시간 = 그리니치 표준시 + 9시
    file_dt = (dt.datetime.now() + dt.timedelta(hours=9)).strftime('%Y%m%d')

    yes_price_file = pd.DataFrame()
    # 예스트레이더 출력용 지표식과 칼럼수 일치시켜야 함.
    yes_price_file = pd.read_csv(path+'yes_stock_price_' + file_dt + '.csv',encoding= 'euc-kr',
                           names = ['날짜1','종목명','종목코드','날짜','시가','고가','저가','종가','거래량',
                                    '단기이평','중기이평','장기이평','지수단기이평','지수중기이평','지수장기이평',
                                    'MACD','MACD_OSC','ATR','CCI','StoK','StoD','TRIX','이격도'],
                           index_col= 1, header=None, dtype={'종목코드':str})

    종목명 = yes_price_file.index.unique()
    종목코드 = yes_price_file['종목코드'].unique()  # 종목코드 사용할 경우를 위해 별도 저장
    df1 = {'종목명':종목명,
           '종목코드':종목코드}
    df1 = pd.DataFrame(df1)
    df1.to_csv('symbol_cd.csv')

    cls_p_data = pd.DataFrame()
    vol_data = pd.DataFrame()
    if gubun == 1:
        hi_p_data = pd.DataFrame()
        lo_p_data = pd.DataFrame()
        sma_data = pd.DataFrame()
        mma_data = pd.DataFrame()
        lma_data = pd.DataFrame()
        sema_data = pd.DataFrame()
        mema_data = pd.DataFrame()
        lema_data = pd.DataFrame()
        macd_data = pd.DataFrame()
        macdo_data = pd.DataFrame()
        atr_data = pd.DataFrame()
        cci_data = pd.DataFrame()
        stok_data = pd.DataFrame()
        stod_data = pd.DataFrame()
        trix_data = pd.DataFrame()
        dis_data = pd.DataFrame()

    for i in range(len(종목명)):
        cls_p = yes_price_file.loc[종목명[i],['날짜','종가']]
        cls_p.set_index(['날짜'],inplace=True)
        cls_p_data = pd.concat([cls_p_data,cls_p],axis=1) 

        vol = yes_price_file.loc[종목명[i],['날짜','거래량']]
        vol.set_index(['날짜'],inplace=True)
        vol_data = pd.concat([vol_data,vol],axis=1) 

        if gubun == 1:
            hi_p = yes_price_file.loc[종목명[i],['날짜','고가']]
            hi_p.set_index(['날짜'],inplace=True)
            hi_p_data = pd.concat([hi_p_data,hi_p],axis=1) 

            lo_p = yes_price_file.loc[종목명[i],['날짜','저가']]
            lo_p.set_index(['날짜'],inplace=True)
            lo_p_data = pd.concat([lo_p_data,lo_p],axis=1) 

            sma = yes_price_file.loc[종목명[i],['날짜','단기이평']]
            sma.set_index(['날짜'],inplace=True)
            sma_data = pd.concat([sma_data,sma],axis=1) 

            mma = yes_price_file.loc[종목명[i],['날짜','중기이평']]
            mma.set_index(['날짜'],inplace=True)
            mma_data = pd.concat([mma_data,mma],axis=1) 

            lma = yes_price_file.loc[종목명[i],['날짜','장기이평']]
            lma.set_index(['날짜'],inplace=True)
            lma_data = pd.concat([lma_data,lma],axis=1) 

            atr = yes_price_file.loc[종목명[i],['날짜','ATR']]
            atr.set_index(['날짜'],inplace=True)
            atr_data = pd.concat([atr_data,atr],axis=1) 

    cls_p_data.columns = 종목명
    print('\n 종가데이터 \n', cls_p_data.head())
    cls_p_data.to_csv('cls_p_data.csv')

    vol_data.columns = 종목명
    print('\n 거래량 \n', vol_data.head())
    vol_data.to_csv('volume_data.csv')

    if gubun == 1:
        hi_p_data.columns = 종목명
        print('\n 고가데이터 \n', hi_p_data.head())
        hi_p_data.to_csv('hi_p_data.csv')

        lo_p_data.columns = 종목명
        print('\n 저가데이터 \n', lo_p_data.head())
        lo_p_data.to_csv('lo_p_data.csv')

        sma_data.columns = 종목명
        print('\n 단기이평 \n', sma_data.head())
        sma_data.to_csv('sma_data.csv')

        mma_data.columns = 종목명
        print('\n 중기이평 \n', mma_data.head())
        mma_data.to_csv('mma_data.csv')

        lma_data.columns = 종목명
        print('\n 장기이평 \n', lma_data.head())
        lma_data.to_csv('lma_data.csv')

        atr_data.columns = 종목명
        print('\n ATR \n', atr_data.head())
        atr_data.to_csv('atr_data.csv')

In [36]:
# 채권ETF가격을 날짜 기준으로 포맷팅
def convert_yes_bond_etf_file():
    # 우리나라 시간 = 그리니치 표준시 + 9시
    file_dt = (dt.datetime.now() + dt.timedelta(hours=9)).strftime('%Y%m%d')

    yes_bond_etf_file = pd.DataFrame()
    yes_bond_etf_file = pd.read_csv(path+'yes_etf_price_' + file_dt + '.csv',encoding= 'euc-kr',
                           names = ['날짜1','종목명','종목코드','날짜','시가','고가','저가','종가','거래량',
                                    '단기이평','중기이평','장기이평','지수단기이평','지수중기이평','지수장기이평',
                                    'MACD','MACD_OSC','ATR','CCI','StoK','StoD','TRIX','이격도'],
                           index_col= 1, header=None, dtype={'종목코드':str})

    종목명2 = yes_bond_etf_file.index.unique()
    종목코드2 = yes_bond_etf_file['종목코드'].unique()  # 종목코드 사용할 경우를 위해 별도 저장
    df2 = {'종목명':종목명2,
           '종목코드':종목코드2}
    df2 = pd.DataFrame(df2)
    df2.to_csv('symbol_cd2.csv')

    out_data2 = pd.DataFrame()
    for i in range(len(종목명2)):
        df2 = yes_bond_etf_file.loc[종목명2[i],['날짜','종가']]
        df2.set_index(['날짜'],inplace=True)
        out_data2 = pd.concat([out_data2,df2],axis=1) 
    out_data2.columns = 종목명2
    print(out_data2.head())
    out_data2.to_csv('bond_etf.csv')

In [37]:
path = '../input/'
gubun = 0 # 지표 파일을 만들려면 1로, 안 만들여면 0
convert_yes_price_file(gubun)
convert_yes_bond_etf_file()


 종가데이터 
 종목명          나노메딕스     필룩스     부산산업   카리스국보    동성제약  에이프로젠 KIC   엔케이물산  \
날짜                                                                       
20151021.0  2585.0  2233.0  32200.0  1508.0  5260.0     3085.0  1226.0   
20151022.0  2560.0  2180.0  31650.0  1546.0  5140.0     3045.0  1193.0   
20151023.0  2650.0  2228.0  31900.0  1496.0  5100.0     3060.0  1217.0   
20151026.0  2560.0  2233.0  33600.0  1442.0  5110.0     3105.0  1197.0   
20151027.0  3100.0  2233.0  33350.0  1521.0  5160.0     3030.0  1209.0   

종목명          웰바이오텍    일신석재     남광토건    삼일제약    인스코비    인디에프       조비   대호에이엘  \
날짜                                                                             
20151021.0  2675.0  1430.0  11050.0  6790.0  1450.0  2420.0  13550.0  1202.0   
20151022.0  2485.0  1340.0  10800.0  6675.0  1395.0  2625.0  13100.0  1159.0   
20151023.0  2590.0  1365.0  11300.0  6733.0  1375.0  2600.0  13600.0  1175.0   
20151026.0  2690.0  1330.0  11150.0  6733.0  1420.0  2660.0  13350.0  1

# 사용자 함수 선언

In [38]:
def getIVP(cov, **kargs):
    # Compute the inverse-variance portfolio
    ivp = 1. / np.diag(cov)  # np.diag - 대각 행렬의 값을 가져옴  
    ivp /= ivp.sum()
    return ivp

In [39]:
def getClusterVar(cov,cItems):
    # Compute variance per cluster
    cov_=cov.loc[cItems,cItems] # matrix slice
    w_=getIVP(cov_).reshape(-1,1)
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar

In [40]:
def getQuasiDiag(link):
    # Sort clustered items by distance
    link = link.astype(int)
    sortIx = pd.Series([link[-1, 0], link[-1, 1]])
    numItems = link[-1, 3]  # number of original items
    while sortIx.max() >= numItems:
        sortIx.index = range(0, sortIx.shape[0] * 2, 2)  # make space
        df0 = sortIx[sortIx >= numItems]  # find clusters
        i = df0.index
        j = df0.values - numItems
        sortIx[i] = link[j, 0]  # item 1
        df0 = pd.Series(link[j, 1], index=i + 1)
        sortIx = sortIx.append(df0)  # item 2
        sortIx = sortIx.sort_index()  # re-sort
        sortIx.index = range(sortIx.shape[0])  # re-index
    return sortIx.tolist()

In [41]:
def getRecBipart(cov, sortIx):
    # Compute HRP alloc
    w = pd.Series(1, index=sortIx)
    cItems = [sortIx]  # initialize all items in one cluster
    while len(cItems) > 0:
        cItems = [i[j:k] for i in cItems for j, k in ((0, len(i) // 2), (len(i) // 2, len(i))) if len(i) > 1]  # bi-section
        for i in range(0, len(cItems), 2):  # parse in pairs
            cItems0 = cItems[i]  # cluster 1
            cItems1 = cItems[i + 1]  # cluster 2
            cVar0 = getClusterVar(cov, cItems0)
            cVar1 = getClusterVar(cov, cItems1)
            alpha = 1 - cVar0 / (cVar0 + cVar1)
            w[cItems0] *= alpha  # weight 1
            w[cItems1] *= 1 - alpha  # weight 2
    return w

In [42]:
def correlDist(corr):
    # A distance matrix based on correlation, where 0<=d[i,j]<=1
    # This is a proper distance metric
    dist = ((1 - corr) / 2.)**.5  # distance matrix
    return dist

In [43]:
def getHRP(cov, corr):
    # Construct a hierarchical portfolio
    dist = correlDist(corr)
    link = sch.linkage(dist, 'single')
    #dn = sch.dendrogram(link, labels=cov.index.values, label_rotation=90)
    #plt.show()
    sortIx = getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist()
    hrp = getRecBipart(cov, sortIx)
    return hrp.sort_index()

In [44]:
def getMVP(cov):

    cov = cov.T.values
    n = len(cov)
    N = 100
    mus = [10 ** (5.0 * t / N - 1.0) for t in range(N)]

    # Convert to cvxopt matrices
    S = opt.matrix(cov)
    #pbar = opt.matrix(np.mean(returns, axis=1))
    pbar = opt.matrix(np.ones(cov.shape[0]))

    # Create constraint matrices
    G = -opt.matrix(np.eye(n))  # negative n x n identity matrix
    h = opt.matrix(0.0, (n, 1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu * S, -pbar, G, h, A, b)['x']
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S * x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']

    return list(wt)

In [45]:
def get_all_portfolios(returns):
    
    cov, corr = returns.cov(), returns.corr() # 공분산과 상관계수
    hrp = getHRP(cov, corr)
    ivp = getIVP(cov)
    ivp = pd.Series(ivp, index=cov.index)
    mvp = getMVP(cov)
    mvp = pd.Series(mvp, index=cov.index)
    
    portfolios = pd.DataFrame([mvp, ivp, hrp], index=['MVP', 'IVP', 'HRP']).T
    
    return portfolios

# Main

In [46]:
# 위험도별 주식비중
Wstk = (0.5, 0.7, 1.0)
공분산_산출기간 = 500  # 시뮬레이터를 통해 최적 값 찾을 것
out_path = '../output/'

# 주식종목 배분
raw_data = pd.read_csv('cls_p_data.csv',index_col=['날짜'], parse_dates=['날짜'])
종목명 = raw_data.columns
noa = len(종목명) # 자산수

symbol_data = pd.read_csv('symbol_cd.csv',index_col=None,dtype={'종목코드':str})
종목코드 = symbol_data['종목코드']
work_data = raw_data.iloc[-공분산_산출기간:]
현재가 = list(raw_data.iloc[-1])
print('\n >> 현재가 :', 현재가)

로그수익률 = np.log(work_data / work_data.shift(1))
print(로그수익률.head())

portfolios = get_all_portfolios(로그수익률)
print(portfolios)

# 우리나라 시간 = 그리니치 표준시 + 9시
current_dt = (dt.datetime.now() + timedelta(hours=9)).strftime('%Y%m%d')
date_col = [current_dt for i in range(noa)] # 날짜 칼럼 생성
print('\n >> 날짜 열 :', date_col)

mp11 = {'날짜':date_col,
      '종목명':종목명,
      '종목코드':종목코드,
      '비중':list((portfolios.HRP*Wstk[0]*100).round(3)),
      '현재가':현재가,
      'MVP':list((portfolios.MVP*Wstk[0]*100).round(3)),
      'IVP':list((portfolios.IVP*Wstk[0]*100).round(3))}
df11 = pd.DataFrame(mp11,columns = ['날짜','종목명','종목코드','비중','현재가','MVP','IVP'])
mp21 = {'날짜':date_col,
      '종목명':종목명,
      '종목코드':종목코드,
      '비중':list((portfolios.HRP*Wstk[1]*100).round(3)),
      '현재가':현재가,
      'MVP':list((portfolios.MVP*Wstk[1]*100).round(3)),
      'IVP':list((portfolios.IVP*Wstk[1]*100).round(3))}
df21 = pd.DataFrame(mp21,columns = ['날짜','종목명','종목코드','비중','현재가','MVP','IVP'])
mp31 = {'날짜':date_col,
      '종목명':종목명,
      '종목코드':종목코드,
      '비중':list((portfolios.HRP*Wstk[2]*100).round(3)),
      '현재가':현재가,
      'MVP':list((portfolios.MVP*Wstk[2]*100).round(3)),
      'IVP':list((portfolios.IVP*Wstk[2]*100).round(3))}
df31 = pd.DataFrame(mp31,columns = ['날짜','종목명','종목코드','비중','현재가','MVP','IVP'])



 >> 현재가 : [9900.0, 9890.0, 171500.0, 3655.0, 17900.0, 2200.0, 755.0, 2500.0, 3005.0, 12750.0, 21500.0, 3295.0, 3070.0, 18150.0, 5210.0, 8220.0, 1150.0, 594.0, 1385.0]
               나노메딕스       필룩스      부산산업     카리스국보      동성제약  에이프로젠 KIC  \
날짜                                                                        
20171103.0       NaN       NaN       NaN       NaN       NaN        NaN   
20171106.0 -0.025508 -0.001761 -0.013226 -0.002020  0.043430  -0.027337   
20171107.0  0.004549 -0.008496 -0.005935 -0.009142  0.001249  -0.009281   
20171108.0  0.006033 -0.008568  0.005935 -0.010257  0.013639   0.011588   
20171109.0  0.120286 -0.010452 -0.005935  0.000000  0.012240   0.092373   

               엔케이물산     웰바이오텍      일신석재      남광토건      삼일제약      인스코비  \
날짜                                                                       
20171103.0       NaN       NaN       NaN       NaN       NaN       NaN   
20171106.0  0.000000  0.025243 -0.007813  0.001496  0.001228 -0.012780   
20171107.0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  after removing the cwd from sys.path.


 3: -9.9994e-01 -9.9998e-01  4e-05  4e-17  3e-05
 4: -9.9995e-01 -9.9996e-01  8e-06  2e-16  5e-06
 5: -9.9995e-01 -9.9995e-01  9e-07  4e-16  1e-07
 6: -9.9995e-01 -9.9995e-01  1e-07  2e-16  8e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.9992e-01 -2.0000e+00  1e+00  2e-15  1e+00
 1: -9.9992e-01 -1.0100e+00  1e-02  2e-16  1e-02
 2: -9.9993e-01 -1.0001e+00  2e-04  5e-17  2e-04
 3: -9.9994e-01 -9.9997e-01  4e-05  1e-16  2e-05
 4: -9.9994e-01 -9.9995e-01  7e-06  8e-17  3e-06
 5: -9.9994e-01 -9.9994e-01  1e-06  4e-17  2e-07
 6: -9.9994e-01 -9.9994e-01  1e-07  2e-16  7e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.9992e-01 -2.0000e+00  1e+00  2e-15  1e+00
 1: -9.9992e-01 -1.0100e+00  1e-02  2e-16  1e-02
 2: -9.9992e-01 -1.0001e+00  2e-04  2e-16  2e-04
 3: -9.9993e-01 -9.9996e-01  4e-05  9e-17  2e-05
 4: -9.9993e-01 -9.9994e-01  6e-06  7e-17  1e-06
 5: -9.9994e-01 -9.9994e-01  9e-07  3e-16  1e-07
 6: -9.9994e-01 -9.9994

 5: -9.8386e-01 -9.8387e-01  5e-06  2e-16  7e-17
 6: -9.8387e-01 -9.8387e-01  4e-07  1e-16  8e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.7915e-01 -1.9997e+00  1e+00  2e-15  1e+00
 1: -9.7935e-01 -1.0086e+00  3e-02  6e-17  3e-02
 2: -9.8110e-01 -9.8588e-01  5e-03  8e-17  3e-03
 3: -9.8176e-01 -9.8229e-01  5e-04  2e-16  8e-17
 4: -9.8188e-01 -9.8193e-01  5e-05  2e-16  9e-17
 5: -9.8190e-01 -9.8190e-01  5e-06  1e-16  7e-17
 6: -9.8190e-01 -9.8190e-01  5e-07  2e-16  8e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.7679e-01 -1.9988e+00  1e+00  8e-16  1e+00
 1: -9.7702e-01 -1.0076e+00  3e-02  5e-17  3e-02
 2: -9.7885e-01 -9.8398e-01  5e-03  6e-17  4e-03
 3: -9.7955e-01 -9.8007e-01  5e-04  1e-16  1e-16
 4: -9.7967e-01 -9.7973e-01  6e-05  9e-17  1e-16
 5: -9.7969e-01 -9.7969e-01  5e-06  5e-17  7e-17
 6: -9.7969e-01 -9.7969e-01  5e-07  2e-16  9e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres


 8:  1.4219e-01  1.4219e-01  7e-09  5e-17  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.7488e-01 -7.7479e-01  2e+01  5e+00  1e+00
 1:  2.8295e-01 -6.3304e-01  1e+00  1e-01  3e-02
 2:  2.9187e-01  1.9874e-01  9e-02  1e-16  4e-16
 3:  2.8344e-01  2.7387e-01  1e-02  2e-16  4e-16
 4:  2.8175e-01  2.8121e-01  5e-04  1e-16  4e-16
 5:  2.8158e-01  2.8152e-01  5e-05  1e-16  2e-16
 6:  2.8156e-01  2.8155e-01  6e-06  8e-17  3e-16
 7:  2.8156e-01  2.8156e-01  5e-07  1e-16  2e-16
 8:  2.8156e-01  2.8156e-01  7e-09  1e-16  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  4.3009e-01 -6.2011e-01  2e+01  5e+00  1e+00
 1:  4.3768e-01 -4.7720e-01  1e+00  1e-01  3e-02
 2:  4.4892e-01  3.4133e-01  1e-01  2e-16  5e-16
 3:  4.3996e-01  4.2920e-01  1e-02  6e-17  3e-16
 4:  4.3814e-01  4.3755e-01  6e-04  2e-16  5e-16
 5:  4.3795e-01  4.3790e-01  6e-05  4e-17  5e-16
 6:  4.3793e-01  4.3793e-01  7e-06  2e-16  3e-16
 7:  4.3793e-01  4.3793

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RankWarning: Polyfit may be poorly conditioned


In [47]:
# 채권ETF 배분
raw_data2 = pd.read_csv('bond_etf.csv',index_col=['날짜'], parse_dates=['날짜'])
종목명2 = raw_data2.columns
print('\n >> 종목명 :', 종목명2)


 >> 종목명 : Index(['KBSTAR 단기통안채', 'KODEX 단기채권', 'KODEX 국고채3년', 'KOSEF 통안채1년'], dtype='object')


In [48]:
noa2 = len(종목명2) # 자산수

symbol_data2 = pd.read_csv('symbol_cd2.csv',index_col=None,dtype={'종목코드':str})
종목코드2 = symbol_data2['종목코드']
print('\n >> 종목코드 :', 종목코드2)


 >> 종목코드 : 0    196230
1    153130
2    114260
3    122260
Name: 종목코드, dtype: object


In [49]:
work_data2 = raw_data2.iloc[-공분산_산출기간:]
현재가2 = list(raw_data2.iloc[-1])
print('\n >> 현재가 :', 현재가2)
로그수익률2 = np.log(work_data2 / work_data2.shift(1))
print(로그수익률2.head())
portfolios2 = get_all_portfolios(로그수익률2)
print(portfolios2)


 >> 현재가 : [104970.0, 101480.0, 56735.0, 102505.0]
            KBSTAR 단기통안채  KODEX 단기채권  KODEX 국고채3년  KOSEF 통안채1년
날짜                                                            
20171103.0           NaN         NaN          NaN          NaN
20171106.0      0.000098    0.000051    -0.000911     0.000243
20171107.0      0.000148    0.000102     0.000818     0.000101
20171108.0      0.000000    0.000051     0.000186     0.000091
20171109.0      0.000138    0.000092    -0.000632     0.000152
     pcost       dcost       gap    pres   dres
 0: -1.0000e+00 -2.0000e+00  1e+00  1e-16  1e+00
 1: -1.0000e+00 -1.0100e+00  1e-02  6e-17  1e-02
 2: -1.0000e+00 -1.0001e+00  1e-04  1e-16  1e-04
 3: -1.0000e+00 -1.0000e+00  1e-06  8e-17  1e-06
 4: -1.0000e+00 -1.0000e+00  3e-08  1e-16  4e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.0000e+00 -2.0000e+00  1e+00  3e-16  1e+00
 1: -1.0000e+00 -1.0100e+00  1e-02  1e-16  1e-02
 2: -1.0000e+00 -1.0001e+00  1e-04  1e-16  1e

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  after removing the cwd from sys.path.


 1: -1.0000e+00 -1.0100e+00  1e-02  1e-16  1e-02
 2: -1.0000e+00 -1.0001e+00  1e-04  1e-16  1e-04
 3: -1.0000e+00 -1.0000e+00  3e-06  1e-16  3e-06
 4: -1.0000e+00 -1.0000e+00  4e-07  1e-16  8e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.0000e+00 -2.0000e+00  1e+00  2e-16  1e+00
 1: -1.0000e+00 -1.0100e+00  1e-02  1e-16  1e-02
 2: -1.0000e+00 -1.0001e+00  1e-04  6e-17  1e-04
 3: -1.0000e+00 -1.0000e+00  3e-06  8e-17  4e-06
 4: -1.0000e+00 -1.0000e+00  4e-07  2e-16  4e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.0000e+00 -2.0000e+00  1e+00  4e-16  1e+00
 1: -1.0000e+00 -1.0100e+00  1e-02  2e-16  1e-02
 2: -1.0000e+00 -1.0001e+00  1e-04  1e-16  1e-04
 3: -1.0000e+00 -1.0000e+00  3e-06  6e-17  4e-06
 4: -1.0000e+00 -1.0000e+00  4e-07  9e-17  5e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.0000e+00 -2.0000e+00  1e+00  8e-17  1e+00
 1: -1.0000e+00 -1.0100e+00  1e-02  8e-17  1e-02


 5: -9.9999e-01 -9.9999e-01  3e-06  6e-17  6e-17
 6: -9.9999e-01 -9.9999e-01  3e-07  1e-16  4e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.9929e-01 -2.0007e+00  1e+00  2e-16  1e+00
 1: -9.9930e-01 -1.0107e+00  1e-02  1e-16  1e-02
 2: -9.9963e-01 -1.0004e+00  8e-04  1e-16  9e-04
 3: -9.9993e-01 -1.0001e+00  2e-04  1e-15  6e-17
 4: -9.9998e-01 -1.0000e+00  2e-05  1e-16  6e-17
 5: -9.9999e-01 -9.9999e-01  3e-06  1e-16  3e-17
 6: -9.9999e-01 -9.9999e-01  4e-07  1e-16  8e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.9921e-01 -2.0008e+00  1e+00  3e-16  1e+00
 1: -9.9922e-01 -1.0108e+00  1e-02  1e-16  1e-02
 2: -9.9960e-01 -1.0005e+00  9e-04  0e+00  1e-03
 3: -9.9993e-01 -1.0001e+00  2e-04  3e-16  3e-17
 4: -9.9998e-01 -1.0000e+00  2e-05  8e-17  5e-17
 5: -9.9999e-01 -9.9999e-01  3e-06  4e-16  9e-17
 6: -9.9999e-01 -9.9999e-01  4e-07  1e-16  4e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RankWarning: Polyfit may be poorly conditioned


# 투자자성향별 MP 비중 조정 후 저장

In [50]:
date_col2 = [current_dt for i in range(noa2)] # 날짜 칼럼 생성

mp12 = {'날짜':date_col2,
      '종목명':종목명2,
      '종목코드':종목코드2,
      '비중':list((portfolios2.HRP*(1-Wstk[0])*100).round(3)),
      '현재가':현재가2,
      'MVP':list((portfolios2.MVP*(1-Wstk[0])*100).round(3)),
      'IVP':list((portfolios2.IVP*(1-Wstk[0])*100).round(3))}
df12 = pd.DataFrame(mp12,columns = ['날짜','종목명','종목코드','비중','현재가','MVP','IVP'])
df11 = pd.concat([df11,df12],axis=0,ignore_index=True)
print(df11)
df11.to_csv(out_path+'HRP_model_portfolio_'+current_dt+'_1.csv', index=None, encoding='ansi')

          날짜           종목명    종목코드      비중       현재가     MVP     IVP
0   20191119         나노메딕스  074610   2.509    9900.0   1.320   1.906
1   20191119           필룩스  033180   2.583    9890.0   2.727   2.046
2   20191119          부산산업  011390   1.837  171500.0   0.191   2.159
3   20191119         카리스국보  001140   2.396    3655.0   2.994   2.227
4   20191119          동성제약  002210   3.043   17900.0   3.451   2.410
5   20191119     에이프로젠 KIC  007460   3.238    2200.0   3.483   2.460
6   20191119         엔케이물산  009810   2.679     755.0   5.267   2.489
7   20191119         웰바이오텍  010600   4.731    2500.0   6.489   2.567
8   20191119          일신석재  007110   0.859    3005.0   0.052   2.707
9   20191119          남광토건  001260   0.873   12750.0   0.445   2.751
10  20191119          삼일제약  000520   3.026   21500.0   3.371   2.739
11  20191119          인스코비  006490   2.303    3295.0   2.870   2.775
12  20191119          인디에프  014990   1.625    3070.0   0.223   2.838
13  20191119            조비  001550

In [51]:
mp22 = {'날짜':date_col2,
      '종목명':종목명2,
      '종목코드':종목코드2,
      '비중':list((portfolios2.HRP*(1-Wstk[1])*100).round(3)),
      '현재가':현재가2,
      'MVP':list((portfolios2.MVP*(1-Wstk[1])*100).round(3)),
      'IVP':list((portfolios2.IVP*(1-Wstk[1])*100).round(3))}
df22 = pd.DataFrame(mp22,columns = ['날짜','종목명','종목코드','비중','현재가','MVP','IVP'])
df21 = pd.concat([df21,df22],axis=0,ignore_index=True) 
print(df21)
df21.to_csv(out_path+'HRP_model_portfolio_'+current_dt+'_2.csv', index=None, encoding='ansi')

          날짜           종목명    종목코드      비중       현재가     MVP     IVP
0   20191119         나노메딕스  074610   3.513    9900.0   1.847   2.669
1   20191119           필룩스  033180   3.617    9890.0   3.818   2.864
2   20191119          부산산업  011390   2.572  171500.0   0.268   3.023
3   20191119         카리스국보  001140   3.355    3655.0   4.192   3.118
4   20191119          동성제약  002210   4.260   17900.0   4.831   3.374
5   20191119     에이프로젠 KIC  007460   4.533    2200.0   4.876   3.444
6   20191119         엔케이물산  009810   3.750     755.0   7.374   3.485
7   20191119         웰바이오텍  010600   6.624    2500.0   9.084   3.594
8   20191119          일신석재  007110   1.203    3005.0   0.073   3.790
9   20191119          남광토건  001260   1.223   12750.0   0.622   3.852
10  20191119          삼일제약  000520   4.237   21500.0   4.720   3.834
11  20191119          인스코비  006490   3.224    3295.0   4.019   3.885
12  20191119          인디에프  014990   2.275    3070.0   0.312   3.973
13  20191119            조비  001550

In [52]:
mp32 = {'날짜':date_col2,
      '종목명':종목명2,
      '종목코드':종목코드2,
      '비중':list((portfolios2.HRP*(1-Wstk[2])*100).round(3)),
      '현재가':현재가2,
      'MVP':list((portfolios2.MVP*(1-Wstk[2])*100).round(3)),
      'IVP':list((portfolios2.IVP*(1-Wstk[2])*100).round(3))}
df32 = pd.DataFrame(mp32,columns = ['날짜','종목명','종목코드','비중','현재가','MVP','IVP'])
df31 = pd.concat([df31,df32],axis=0,ignore_index=True) 
print(df31)
df31.to_csv(out_path +'HRP_model_portfolio_'+current_dt+'_3.csv', index=None, encoding='ansi')

          날짜           종목명    종목코드      비중       현재가     MVP    IVP
0   20191119         나노메딕스  074610   5.018    9900.0   2.639  3.813
1   20191119           필룩스  033180   5.166    9890.0   5.455  4.092
2   20191119          부산산업  011390   3.674  171500.0   0.383  4.318
3   20191119         카리스국보  001140   4.793    3655.0   5.989  4.454
4   20191119          동성제약  002210   6.086   17900.0   6.901  4.820
5   20191119     에이프로젠 KIC  007460   6.476    2200.0   6.965  4.921
6   20191119         엔케이물산  009810   5.357     755.0  10.534  4.979
7   20191119         웰바이오텍  010600   9.463    2500.0  12.977  5.134
8   20191119          일신석재  007110   1.718    3005.0   0.104  5.414
9   20191119          남광토건  001260   1.746   12750.0   0.889  5.502
10  20191119          삼일제약  000520   6.053   21500.0   6.743  5.477
11  20191119          인스코비  006490   4.606    3295.0   5.741  5.551
12  20191119          인디에프  014990   3.250    3070.0   0.446  5.676
13  20191119            조비  001550   3.301   181